## Algoritmo LSTM

In [9]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from collections import Counter

In [2]:
nombre_csv = 1
de = pd.read_csv(f'Datos_entrenamiento/{nombre_csv}_e.csv')
dp = pd.read_csv(f'Datos_prueba/{nombre_csv}_p.csv')

# Procesamiento

In [3]:
# Contamos las ocurrencias de cada nivel de estrés en la nueva columna
conteo_nivel_estresde = de['Nivel_estres_2'].value_counts()
conteo_nivel_estresdp = dp['Nivel_estres_2'].value_counts()
# Mostramos el resultado
print(conteo_nivel_estresde)
print(conteo_nivel_estresdp)

Nivel_estres_2
normal     3600
alerta      288
peligro      48
Name: count, dtype: int64
Nivel_estres_2
normal     2448
alerta       24
peligro      24
Name: count, dtype: int64


In [4]:
# Crear un diccionario para mapear cada categoría única en la columna 'nivel_estres' a un número
mapping = {'normal': 0, 'alerta': 1, 'peligro': 2}
# Mapear las categorías a números utilizando el método map de pandas
de['Nivel_estres_1'] = de['Nivel_estres_1'].map(mapping)
dp['Nivel_estres_1'] = dp['Nivel_estres_1'].map(mapping)
de['Nivel_estres_2'] = de['Nivel_estres_2'].map(mapping)
dp['Nivel_estres_2'] = dp['Nivel_estres_2'].map(mapping)

# ALGORITMO DE LSTM

In [5]:
# Separar características y etiquetas del conjunto de entrenamiento
X_train = de[['period eating','period other','period resting','period rumination']]
y_train = de['Nivel_estres_1']
# Separar características y etiquetas del conjunto de prueba
X_test = dp[['period eating','period other','period resting','period rumination']]
y_test = dp['Nivel_estres_1']

In [6]:
# # Separar características y etiquetas del conjunto de entrenamiento
# X_train = de[['period eating_count','period other_count','period resting_count','period rumination_count']]
# y_train = de['Nivel_estres_2']
# # Separar características y etiquetas del conjunto de prueba
# X_test = dp[['period eating_count','period other_count','period resting_count','period rumination_count']]
# y_test = dp['Nivel_estres_2']

In [7]:
# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Convertir etiquetas a formato categórico
y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)

In [8]:
# Reshape de datos para LSTM (samples, timesteps, features)
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


In [ ]:
# Construir el modelo LSTM
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(24, X_train_scaled.shape[24])))
model.add(LSTM(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['auc'])


C:\Users\PC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Entrenar el modelo
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=24, validation_data=(X_test_reshaped, y_test_categorical))


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - auc: 0.7378 - loss: 1.0051 - val_auc: 0.8468 - val_loss: 0.7283
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - auc: 0.7959 - loss: 0.8152 - val_auc: 0.8553 - val_loss: 0.7002
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.8022 - loss: 0.8051 - val_auc: 0.8579 - val_loss: 0.6923
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.8142 - loss: 0.7843 - val_auc: 0.8587 - val_loss: 0.6912
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - auc: 0.8090 - loss: 0.7914 - val_auc: 0.8590 - val_loss: 0.6859
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - auc: 0.8072 - loss: 0.8049 - val_auc: 0.8593 - val_loss: 0.6809
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - auc: 0.8141 - loss: 0.7892 - val_auc: 0.8596 - val_loss: 0.6847
Epoch 8/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - auc: 0.8117 - loss: 0.7875 - val_auc: 0.8593 - val_loss: 0.6831
Epoch 9/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - au

In [11]:
# Predecir las etiquetas para el conjunto de prueba
y_pred_proba = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_proba, axis=1)

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [12]:
# Contamos las ocurrencias de cada nivel de estrés
valores_unicos, conteos = np.unique(y_test, return_counts=True)
valores_unicos1, conteos1 = np.unique(y_pred, return_counts=True)
# Combina los valores únicos y sus conteos en un diccionario para facilitar la visualización
ocurrencias = dict(zip(valores_unicos, conteos))
ocurrenciaspre = dict(zip(valores_unicos1, conteos1))
# Mostramos el resultado
#print(conteo_nivel_estres)
print(ocurrencias)
print(ocurrenciaspre)

{0: 1896, 1: 480, 2: 120}
{0: 2496}


# EVALUACION DEL ALGORITMO

In [10]:
# EVALUACION DEL MODELO
from sklearn.preprocessing import label_binarize

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])

# Calcular el AUC
roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovo')

# Mostrar métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

ValueError: Found input variables with inconsistent numbers of samples: [2472, 2496]

In [11]:
fpr = {}
tpr = {}
roc_auc = {}

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Graficar las curvas ROC
plt.figure()
for i in range(3):
    plt.plot(fpr[i], tpr[i], label=f'Clase {i} (AUC = {roc_auc[i]:.2f})')

# Extraer los valores
values = roc_auc.values()
# Calcular la media
mean_auc = sum(values) / len(values)

plt.plot([0, 1], [0, 1], 'b--', label=f"Mean AUC = {mean_auc:.2f}", alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC Multiclase')
plt.legend(loc='lower right')
#plt.savefig(f'../Curva_Roc/{nombre_csv}_roc_LSTM.png', format='png', dpi=300)
plt.show()

NameError: name 'y_test_binarized' is not defined

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

nombre_csv = 1
de = pd.read_csv(f'../Datos_entrenamiento/{nombre_csv}_e.csv')
dp = pd.read_csv(f'../Datos_prueba/{nombre_csv}_p.csv')

mapping = {'normal': 0, 'alerta': 1, 'peligro': 2}
de['Nivel_estres_1'] = de['Nivel_estres_1'].map(mapping)
dp['Nivel_estres_1'] = dp['Nivel_estres_1'].map(mapping)

X_train = de[['period eating', 'period other', 'period resting', 'period rumination']]
y_train = de['Nivel_estres_1']
X_test = dp[['period eating', 'period other', 'period resting', 'period rumination']]
y_test = dp['Nivel_estres_1']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Expandir a 24 horas de datos para capturar ciclo diario
timesteps = 24
X_train_reshaped = np.array([X_train_scaled[i:i + timesteps] for i in range(len(X_train_scaled) - timesteps)])
y_train = y_train[timesteps:]
X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
y_test = y_test[timesteps:]
#Ajuste de pesos
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)

model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(timesteps, X_train.shape[1])))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['auc'])

model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=24,
          validation_data=(X_test_reshaped, y_test_categorical), class_weight=class_weights_dict)

test_loss, test_auc = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"AUC en prueba: {test_auc}")

C:\Users\PC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - auc: 0.5088 - loss: 1.1010 - val_auc: 0.8681 - val_loss: 0.7872
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - auc: 0.7520 - loss: 0.9907 - val_auc: 0.8542 - val_loss: 0.8848
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - auc: 0.7812 - loss: 0.9319 - val_auc: 0.8403 - val_loss: 0.8114
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - auc: 0.8007 - loss: 0.9078 - val_auc: 0.8483 - val_loss: 0.7932
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - auc: 0.8198 - loss: 0.8440 - val_auc: 0.8559 - val_loss: 0.8166
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - auc: 0.8192 - loss: 0.8417 - val_auc: 0.8508 - val_loss: 0.8149
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - auc: 0.8364 - loss: 0.8097 - val_auc: 0.8460 - val_loss: 0.8402
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - auc: 0.8236 - loss: 0.8166 - val_auc: 0.8591 - val_loss: 0.7372
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/s

In [3]:
# Predecir las etiquetas para el conjunto de prueba
y_pred_proba = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_proba, axis=1)

# Contamos las ocurrencias de cada nivel de estrés
valores_unicos, conteos = np.unique(y_test, return_counts=True)
valores_unicos1, conteos1 = np.unique(y_pred, return_counts=True)
# Combina los valores únicos y sus conteos en un diccionario para facilitar la visualización
ocurrencias = dict(zip(valores_unicos, conteos))
ocurrenciaspre = dict(zip(valores_unicos1, conteos1))
# Mostramos el resultado
#print(conteo_nivel_estres)
print(ocurrencias)
print(ocurrenciaspre)

78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
{0: 1896, 1: 456, 2: 120}
{0: 1935, 1: 480, 2: 57}


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

# 1. Mejora en el preprocesamiento de datos
def prepare_data(df_train, df_test):
    # Características y etiquetas
    features = ['period eating', 'period other', 'period resting', 'period rumination']
    X_train = df_train[features]
    y_train = df_train['Nivel_estres_1']
    X_test = df_test[features]
    y_test = df_test['Nivel_estres_1']
    
    # Estandarización
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Conversión a formato LSTM
    X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
    
    # Conversión de etiquetas
    y_train_categorical = to_categorical(y_train, num_classes=3)
    y_test_categorical = to_categorical(y_test, num_classes=3)
    
    return X_train_reshaped, y_train_categorical, X_test_reshaped, y_test_categorical, y_train

# 2. Modelo LSTM mejorado
def create_improved_model(input_shape):
    model = Sequential([
        LSTM(128, activation='relu', input_shape=input_shape, return_sequences=True),
        LSTM(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']
    )
    
    return model

# 3. Función principal de entrenamiento
def train_stress_model(nombre_csv):
    # Cargar datos
    de = pd.read_csv(f'Datos_entrenamiento/{nombre_csv}_e.csv')
    dp = pd.read_csv(f'Datos_prueba/{nombre_csv}_p.csv')
    
    # Mapear niveles de estrés
    mapping = {'normal': 0, 'alerta': 1, 'peligro': 2}
    de['Nivel_estres_1'] = de['Nivel_estres_1'].map(mapping)
    dp['Nivel_estres_1'] = dp['Nivel_estres_1'].map(mapping)
    
    # Preparar datos
    X_train_reshaped, y_train_categorical, X_test_reshaped, y_test_categorical, y_train = prepare_data(de, dp)
    
    # Calcular pesos de clase para manejar el desbalance
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(enumerate(class_weights))
    
    # Crear y entrenar modelo
    model = create_improved_model((1, X_train_reshaped.shape[2]))
    
    history = model.fit(
        X_train_reshaped, 
        y_train_categorical,
        epochs=100,
        batch_size=32,
        validation_data=(X_test_reshaped, y_test_categorical),
        class_weight=class_weight_dict,
        shuffle=True
    )
    
    return model, history

# 4. Evaluación y predicción
def evaluate_model(model, X_test_reshaped, y_test_categorical):
    # Evaluación
    evaluation = model.evaluate(X_test_reshaped, y_test_categorical)
    
    # Predicciones
    y_pred_proba = model.predict(X_test_reshaped)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    return evaluation, y_pred, y_pred_proba

# Uso del código mejorado
if __name__ == "__main__":
    nombre_csv = 1
    model, history = train_stress_model(nombre_csv)
    
    # Cargar datos de prueba para evaluación
    dp = pd.read_csv(f'Datos_prueba/{nombre_csv}_p.csv')
    mapping = {'normal': 0, 'alerta': 1, 'peligro': 2}
    dp['Nivel_estres_1'] = dp['Nivel_estres_1'].map(mapping)
    
    # Preparar datos de prueba
    _, _, X_test_reshaped, y_test_categorical, _ = prepare_data(dp, dp)
    
    # Evaluar modelo
    evaluation, y_pred, y_pred_proba = evaluate_model(model, X_test_reshaped, y_test_categorical)
    
    # Imprimir resultados
    print(f"Métricas de evaluación: {evaluation}")
    print(f"Distribución de predicciones: {np.bincount(y_pred)}")

C:\Users\PC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - AUC: 0.4493 - accuracy: 0.2251 - loss: 1.1105 - val_AUC: 0.5126 - val_accuracy: 0.5489 - val_loss: 1.0988
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - AUC: 0.4749 - accuracy: 0.3783 - loss: 1.0927 - val_AUC: 0.5958 - val_accuracy: 0.5509 - val_loss: 1.0996
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5044 - accuracy: 0.4361 - loss: 1.0973 - val_AUC: 0.6423 - val_accuracy: 0.5853 - val_loss: 1.0828
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5981 - accuracy: 0.5225 - loss: 1.0737 - val_AUC: 0.7195 - val_accuracy: 0.6442 - val_loss: 1.0634
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.6157 - accuracy: 0.5450 - loss: 1.0923 - val_AUC: 0.6912 - val_accuracy: 0.6214 - val_loss: 1.0678
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5890 - accuracy: 0.5198 - loss: 1.0957 - val_AUC: 0.6951 - val_accuracy: 0.6262 - val_loss: 1.0730
Epoch 7/100
123/123 ━━━━━━━━━━━━━

In [12]:
history